## Umbral densidad poblacional

En este notebook clasificaremos los municipios como despoblados o poblados siguiendo la propia definición de despoblación: municipios con una densidad de población inferior a 12,5 habitantes/km2.  
Además, también crearemos dos DataFrame que utilizaremos posteriormente para hacer los mapas coropléticos interactivos de la península(incluyendo Baleares, Ceuta y Melilla) y Canarias.

In [1]:
import pandas as pd
import numpy as np

Cargamos las tablas de densidad de población de 2011 y 2021:

In [2]:
Densidad11 = pd.read_csv('/home/dsc/Python/Despoblacion/2.Data/03_Output_Densidad_Poblacion_11.csv' ,
                         dtype = {'Codigo Municipio' : str })

Densidad21 = pd.read_csv('/home/dsc/Python/Despoblacion/2.Data/03_Output_Densidad_Poblacion_21.csv' ,
                         dtype = {'Codigo Municipio' : str })

In [3]:
Densidad11.head()

,Codigo Municipio,Superficie Km2,Densidad Poblacion
0,01001,19.95,140.50
1,01002,96.30,103.99
2,01003,73.02,20.49
3,01004,27.29,67.53
4,01006,12.97,17.66


In [4]:
Densidad21.head()

,Codigo Municipio,Superficie Km2,Densidad Poblacion
0,01001,19.95,146.62
1,01002,96.30,107.03
2,01003,73.02,19.94
3,01004,27.29,66.29
4,01006,12.97,17.81


Creamos una nueva columna que clasifica a los municipios en despoblados; < 12,5 hab/km2 o poblados; >=12,5 hab/km2

In [5]:
Densidad11['Umbral'] = Densidad11['Densidad Poblacion'].map(lambda x: 'Despoblado' if x < 12.5 else 'Poblado')
Densidad21['Umbral'] = Densidad21['Densidad Poblacion'].map(lambda x: 'Despoblado' if x < 12.5 else 'Poblado')

In [6]:
Densidad11['Umbral'].value_counts()

Poblado       4411
Despoblado    3705
Name: Umbral, dtype: int64

In [7]:
Densidad21['Umbral'].value_counts()

Poblado       4184
Despoblado    3947
Name: Umbral, dtype: int64

Juntamos los dos DataFrame y creamos dos nuevas columas:
- **Densidad**: Si el municipio en 2011 ya está despoblado, toma el valor de la densidad de población de 2011, si está poblado, toma el valor de densidad de 2021
- **Umbral**: Si está despoblado en 2011, toma el valor 'Despoblado11', si está poblado en 2011 y en 2021 pasa a ser despoblado, toma el valor 'Despoblado21', el resto de casos toman el valor 'Poblado'.  

A tener en cuenta los np.nan como consecuencia de las modificaciones de municipios de 2021 respecto de 2011:

In [8]:
Densidad = Densidad21. merge(Densidad11, on = 'Codigo Municipio', how = 'left', suffixes = ('_21', '_11' ))

Densidad.head()

,Codigo Municipio,Superficie Km2_21,Densidad Poblacion_21,Umbral_21,Superficie Km2_11,Densidad Poblacion_11,Umbral_11
0,01001,19.95,146.62,Poblado,19.95,140.50,Poblado
1,01002,96.30,107.03,Poblado,96.30,103.99,Poblado
2,01003,73.02,19.94,Poblado,73.02,20.49,Poblado
3,01004,27.29,66.29,Poblado,27.29,67.53,Poblado
4,01006,12.97,17.81,Poblado,12.97,17.66,Poblado


In [9]:
Densidad['Densidad'] = np.where(Densidad['Umbral_11'] == 'Despoblado', 
                                Densidad['Densidad Poblacion_11'], 
                                Densidad['Densidad Poblacion_21'] )

In [10]:
Densidad.loc[(Densidad['Umbral_11'] == 'Despoblado'), 'Umbral'] = 'Despoblado11' 

Densidad.loc[((Densidad['Umbral_11'] == 'Poblado') | (Densidad['Umbral_11'].isnull())), 
             'Umbral'] = 'Despoblado21'

Densidad.loc[((Densidad['Umbral_11'] == 'Poblado') | (Densidad['Umbral_11'].isnull())) 
             & (Densidad['Umbral_21'] == 'Poblado'), 'Umbral'] = 'Poblado'

In [11]:
Densidad.head()

,Codigo Municipio,Superficie Km2_21,Densidad Poblacion_21,Umbral_21,Superficie Km2_11,Densidad Poblacion_11,Umbral_11,Densidad,Umbral
0,01001,19.95,146.62,Poblado,19.95,140.50,Poblado,146.62,Poblado
1,01002,96.30,107.03,Poblado,96.30,103.99,Poblado,107.03,Poblado
2,01003,73.02,19.94,Poblado,73.02,20.49,Poblado,19.94,Poblado
3,01004,27.29,66.29,Poblado,27.29,67.53,Poblado,66.29,Poblado
4,01006,12.97,17.81,Poblado,12.97,17.66,Poblado,17.81,Poblado


Creamos un DataFrame solo con municipios pertenecientes a Canarias y otro DataFrame con el resto de municipios de España.  
Para realizar dicha segmentación cargamos la tabla 01_Output_ProvMun_21.csv que contiene el campo Nombre Provincia o Codigo Provincia y hacemos un merge:

In [12]:
ProvMun = pd.read_csv('/home/dsc/Python/Despoblacion/2.Data/01_Output_ProvMun_21.csv' ,
                      dtype = {'Codigo CCAA' : str, 'Codigo Municipio' : str, 'Codigo Provincia' : str})

In [13]:
Densidad = Densidad.merge(ProvMun, on = 'Codigo Municipio')

Densidad.head()

,Codigo Municipio,Superficie Km2_21,Densidad Poblacion_21,Umbral_21,Superficie Km2_11,Densidad Poblacion_11,Umbral_11,Densidad,Umbral,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio
0,01001,19.95,146.62,Poblado,19.95,140.50,Poblado,146.62,Poblado,Pais Vasco,16,Araba/Álava,01,Alegría-Dulantzi
1,01002,96.30,107.03,Poblado,96.30,103.99,Poblado,107.03,Poblado,Pais Vasco,16,Araba/Álava,01,Amurrio
2,01003,73.02,19.94,Poblado,73.02,20.49,Poblado,19.94,Poblado,Pais Vasco,16,Araba/Álava,01,Aramaio
3,01004,27.29,66.29,Poblado,27.29,67.53,Poblado,66.29,Poblado,Pais Vasco,16,Araba/Álava,01,Artziniega
4,01006,12.97,17.81,Poblado,12.97,17.66,Poblado,17.81,Poblado,Pais Vasco,16,Araba/Álava,01,Armiñón


In [14]:
Canarias = Densidad[Densidad['Nombre CCAA'] == 'Canarias']

Canarias.head()

,Codigo Municipio,Superficie Km2_21,Densidad Poblacion_21,Umbral_21,Superficie Km2_11,Densidad Poblacion_11,Umbral_11,Densidad,Umbral,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio
5250,35001,44.51,127.95,Poblado,44.51,129.77,Poblado,127.95,Poblado,Canarias,05,"Palmas, Las",35,Agaete
5251,35002,78.90,406.91,Poblado,78.90,377.28,Poblado,406.91,Poblado,Canarias,05,"Palmas, Las",35,Agüimes
5252,35003,250.25,51.08,Poblado,250.25,42.36,Poblado,51.08,Poblado,Canarias,05,"Palmas, Las",35,Antigua
5253,35004,24.28,2656.38,Poblado,24.28,2362.31,Poblado,2656.38,Poblado,Canarias,05,"Palmas, Las",35,Arrecife
5254,35005,66.42,15.75,Poblado,66.42,18.99,Poblado,15.75,Poblado,Canarias,05,"Palmas, Las",35,Artenara


In [15]:
Peninsula = Densidad[Densidad['Nombre CCAA'] != 'Canarias']

Peninsula.head()

,Codigo Municipio,Superficie Km2_21,Densidad Poblacion_21,Umbral_21,Superficie Km2_11,Densidad Poblacion_11,Umbral_11,Densidad,Umbral,Nombre CCAA,Codigo CCAA,Nombre Provincia,Codigo Provincia,Nombre Municipio
0,01001,19.95,146.62,Poblado,19.95,140.50,Poblado,146.62,Poblado,Pais Vasco,16,Araba/Álava,01,Alegría-Dulantzi
1,01002,96.30,107.03,Poblado,96.30,103.99,Poblado,107.03,Poblado,Pais Vasco,16,Araba/Álava,01,Amurrio
2,01003,73.02,19.94,Poblado,73.02,20.49,Poblado,19.94,Poblado,Pais Vasco,16,Araba/Álava,01,Aramaio
3,01004,27.29,66.29,Poblado,27.29,67.53,Poblado,66.29,Poblado,Pais Vasco,16,Araba/Álava,01,Artziniega
4,01006,12.97,17.81,Poblado,12.97,17.66,Poblado,17.81,Poblado,Pais Vasco,16,Araba/Álava,01,Armiñón


In [16]:
NueCol = ['Codigo Municipio', 'Densidad', 'Umbral']

In [17]:
Canarias = Canarias[NueCol]
Peninsula = Peninsula[NueCol]

In [18]:
Canarias.to_csv('Canarias.csv', header = True, index = False)

In [19]:
Peninsula.to_csv('Peninsula.csv', header = True, index = False)